In [60]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import asyncio
from pydantic import BaseModel,Field
from autogen_ext.tools.http import HttpTool
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.base import TaskResult



# Create the token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-08-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)


In [49]:
## Agent definition
# Define a simple tool that can be used by the agent
dsa_solver_agent = AssistantAgent(
    name="dsa_solver",
    description="A DSA solver agent that can answer questions related to data structures and algorithms.",
    system_message= " You give code snippets to solve DSA problems. You are a DSA expert." ,
    model_client=az_model_client
)


code_reviewer_agent = AssistantAgent(
    name="code_reviewer",
    description="A code reviewer agent that can review code snippets and provide feedback.",
    system_message="You are a code reviewer. You review code snippets and provide feedback.",   
    model_client=az_model_client
)


code_editor_agent = AssistantAgent(
    name="code_editor",
    description="A code editor agent that can edit code snippets and provide suggestions and Add comments. Give under 10 lines comments",
    system_message="You are a code editor. You edit code snippets and provide suggestions. If all looks good, you can say 'TERMINATE'.",
    model_client=az_model_client
)



In [62]:
agent_team = [dsa_solver_agent, code_reviewer_agent, code_editor_agent]
team = RoundRobinGroupChat(participants=agent_team,max_turns= 30,termination_condition=TextMentionTermination("TERMINATE"))

In [ ]:
team.run_stream(task="Write a Python function to add 2 numbers.")

<async_generator object BaseGroupChat.run_stream at 0x000002E159AE60C0>


In [55]:
async def run_team_chat(user_prompt):
    text_message = TextMessage(content=user_prompt, source="User")
    response = await team.run(task=text_message)
    for message in response.messages:
        print(f"{message.source }")
        print("--------------------------------------------------")
        print("Team chat completed successfully.")
        print("--------------------------------------------------")
        

# Example usage:
await run_team_chat("Write a Python function to add 2 numbers.")


User
--------------------------------------------------
Team chat completed successfully.
--------------------------------------------------
dsa_solver
--------------------------------------------------
Team chat completed successfully.
--------------------------------------------------
code_reviewer
--------------------------------------------------
Team chat completed successfully.
--------------------------------------------------
code_editor
--------------------------------------------------
Team chat completed successfully.
--------------------------------------------------


In [73]:
async def run_team_chat(user_prompt):
    text_message = TextMessage(content=user_prompt, source="User")
    async for message in team.run_stream(task=text_message):
        if isinstance(message, TaskResult):
           print(f"Task Result: {message.stop_reason}")
        else:
            print(f"{message.source}")
        print("--------------------------------------------------")

# Example usage:
await run_team_chat("Write a Python function to add 2 numbers.")


User
--------------------------------------------------
dsa_solver
--------------------------------------------------
code_reviewer
--------------------------------------------------
code_editor
--------------------------------------------------
Task Result: Text 'TERMINATE' mentioned
--------------------------------------------------


In [ ]:
# The error in cell 6 is that run_team_chat() requires a user_prompt argument.
# Provide a prompt string when calling the function.

await run_team_chat("Write a Python program to check if a number is even.")

In [33]:
await run_team_chat()

User
--------------------------------------------------
dsa_solver
--------------------------------------------------
code_reviewer
--------------------------------------------------
code_editor
--------------------------------------------------


In [ ]:
Team Response: messages=[
    
    
    TextMessage(id='40c64f35-a39b-4984-90e0-cbef7573b2db', source='User', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 32, 905088, tzinfo=datetime.timezone.utc), content='Write a Python program to check if a number is even.', type='TextMessage'), 
    
    TextMessage(id='67e7e1de-2a8c-48dc-b3c3-40564a2e934e', source='dsa_solver', models_usage=RequestUsage(prompt_tokens=183, completion_tokens=221), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 35, 989842, tzinfo=datetime.timezone.utc), content='Certainly! Here’s a simple Python program to check if a number is even:\n\n```python\ndef is_even(number):\n    return number % 2 == 0\n\n# Input from the user\ntry:\n    user_input = int(input("Enter a number: "))\n    if is_even(user_input):\n        print(f"{user_input} is an even number.")\n    else:\n        print(f"{user_input} is an odd number.")\nexcept ValueError:\n    print("Please enter a valid integer.")\n```\n\n### How It Works:\n1. The function `is_even` takes a number as input and returns `True` if the number is even (i.e., divisible by 2 with no remainder), and `False` otherwise.\n2. The program prompts the user to enter a number.\n3. It checks whether the input is a valid integer, and then calls the `is_even` function to determine if the number is even or odd, printing the appropriate message to the user.\n\nFeel free to run this code and let me know if you have any questions or modifications in mind!', type='TextMessage'), 
    
    
    TextMessage(id='606eeca5-795b-4130-8b8d-751604f302ac', source='code_reviewer', models_usage=RequestUsage(prompt_tokens=406, completion_tokens=553), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 45, 394097, tzinfo=datetime.timezone.utc), content='Your code for checking if a number is even is well-structured and effectively demonstrates the concept. Here are a few suggestions and feedback to consider for improvement:\n\n### Strengths:\n1. **Functionality**: The program correctly implements the check for even and odd numbers.\n2. **Error Handling**: You\'ve included error handling for invalid inputs, which is an important aspect of user-friendly applications.\n3. **Readability**: The code is easy to read and understand, thanks to clear function and variable names.\n\n### Suggestions for Improvement:\n1. **Docstrings**: Consider adding a docstring to the `is_even` function to describe its purpose, parameters, and return value. This improves code documentation.\n\n   ```python\n   def is_even(number):\n       """\n       Check if a number is even.\n\n       Parameters:\n       number (int): The number to check.\n\n       Returns:\n       bool: True if the number is even, False otherwise.\n       """\n       return number % 2 == 0\n   ```\n\n2. **Integer Input Validation**: Although the `try-except` block handles invalid input, you may want to loop back to ask for input again instead of terminating the program. This makes it more user-friendly.\n\n    ```python\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n    ```\n\n3. **Function Modularity**: While your code is concise, you could separate the input and output logic from the checking logic for better modularity.\n\n### Revised Code Example:\nHere\'s an updated version of your program incorporating these suggestions:\n\n```python\ndef is_even(number):\n    """\n    Check if a number is even.\n\n    Parameters:\n    number (int): The number to check.\n\n    Returns:\n    bool: True if the number is even, False otherwise.\n    """\n    return number % 2 == 0\n\ndef main():\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n    \n    if is_even(user_input):\n        print(f"{user_input} is an even number.")\n    else:\n        print(f"{user_input} is an odd number.")\n\nif __name__ == "__main__":\n    main()\n```\n\n### Conclusion:\nYour initial code is great, and these enhancements will make it even more robust, user-friendly, and maintainable. If you have any further questions or specific areas you\'d like to explore, feel free to ask!', type='TextMessage'), 
    
    
    TextMessage(id='b15bca37-9f87-4076-9c13-e36c7e396b99', source='code_editor', models_usage=RequestUsage(prompt_tokens=967, completion_tokens=552), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 49, 674140, tzinfo=datetime.timezone.utc), content='Your code for checking if a number is even is well-structured and effectively demonstrates the concept. Here are a few suggestions and feedback to consider for improvement:\n\n### Strengths:\n1. **Functionality**: The program correctly implements the check for even and odd numbers.\n2. **Error Handling**: You\'ve included error handling for invalid inputs, which is an important aspect of user-friendly applications.\n3. **Readability**: The code is easy to read and understand, thanks to clear function and variable names.\n\n### Suggestions for Improvement:\n1. **Docstrings**: Consider adding a docstring to the `is_even` function to describe its purpose, parameters, and return value. This improves code documentation.\n\n   ```python\n   def is_even(number):\n       """\n       Check if a number is even.\n\n       Parameters:\n       number (int): The number to check.\n\n       Returns:\n       bool: True if the number is even, False otherwise.\n       """\n       return number % 2 == 0\n   ```\n\n2. **Integer Input Validation**: Although the `try-except` block handles invalid input, you may want to loop back to ask for input again instead of terminating the program. This makes it more user-friendly.\n\n    ```python\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n    ```\n\n3. **Function Modularity**: While your code is concise, you could separate the input and output logic from the checking logic for better modularity.\n\n### Revised Code Example:\nHere\'s an updated version of your program incorporating these suggestions:\n\n```python\ndef is_even(number):\n    """\n    Check if a number is even.\n\n    Parameters:\n    number (int): The number to check.\n\n    Returns:\n    bool: True if the number is even, False otherwise.\n    """\n    return number % 2 == 0\n\ndef main():\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n\n    if is_even(user_input):\n        print(f"{user_input} is an even number.")\n    else:\n        print(f"{user_input} is an odd number.")\n\nif __name__ == "__main__":\n    main()\n```\n\n### Conclusion:\nYour initial code is great, and these enhancements will make it even more robust, user-friendly, and maintainable. If you have any further questions or specific areas you\'d like to explore, feel free to ask!', type='TextMessage')] 


stop_reason='Maximum number of turns 3 reached.'